In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test  = x_test[..., tf.newaxis].astype("float32")

print("x_train shape:", x_train.shape)
print("x_test shape :", x_test.shape)

batch_size = 32

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
    .shuffle(10000) \
    .batch(batch_size)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
    .batch(batch_size)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
x_train shape: (60000, 28, 28, 1)
x_test shape : (10000, 28, 28, 1)


In [2]:

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1   = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1      = Dense(128, activation='relu')
        self.d2      = Dense(10)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()


In [3]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss      = tf.keras.metrics.Mean(name='train_loss')
train_accuracy  = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss       = tf.keras.metrics.Mean(name='test_loss')
test_accuracy   = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [4]:

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_accuracy.update_state(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss.update_state(t_loss)
    test_accuracy.update_state(labels, predictions)


In [5]:

EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss.reset_state()
    train_accuracy.reset_state()
    test_loss.reset_state()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = (
        "Epoch {}, Loss: {:.4f}, Accuracy: {:.2f}%, "
        "Test Loss: {:.4f}, Test Accuracy: {:.2f}%"
    )
    print(template.format(
        epoch + 1,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100
    ))

print("\n✅ Entrenamiento completado.")


Epoch 1, Loss: 0.1390, Accuracy: 95.86%, Test Loss: 0.0688, Test Accuracy: 97.74%
Epoch 2, Loss: 0.0435, Accuracy: 98.64%, Test Loss: 0.0511, Test Accuracy: 98.38%
Epoch 3, Loss: 0.0224, Accuracy: 99.25%, Test Loss: 0.0540, Test Accuracy: 98.34%
Epoch 4, Loss: 0.0145, Accuracy: 99.49%, Test Loss: 0.0559, Test Accuracy: 98.41%
Epoch 5, Loss: 0.0093, Accuracy: 99.68%, Test Loss: 0.0635, Test Accuracy: 98.35%

✅ Entrenamiento completado.
